In [1]:
import requests
import pandas as pd
from time import sleep
from random import randint
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle

In [2]:
df2 = pd.read_csv('songclusters.csv', index_col=0)
df2.head(10)

,artists,track_name,clusters
0,Queen,Bohemian Rhapsody - Remastered 2011,1
1,Deep Purple,Child in Time,2
2,Meat Loaf,Paradise By the Dashboard Light,7
3,The Moody Blues,Nights In White Satin - Single Version / Mono,2
4,Aphrodite's Child,Rain And Tears,1
5,Eagles,Hotel California - 2013 Remaster,4
6,Elvis Presley,Suspicious Minds,4
7,Procol Harum,A Whiter Shade Of Pale,7
8,The Beatles,Yesterday - Remastered 2009,1
9,Barry White,"You're The First, The Last, My Everything - Edit",4


In [3]:
df1 = pd.read_csv('datasongs.csv')
df1.head()

,title,artist
0,Rich Men North of Richmond,Oliver Anthony Music
1,Margaritaville,Jimmy Buffett
2,Come Monday,Jimmy Buffett
3,I Remember Everything (feat. Kacey Musgraves),Zach Bryan
4,Paint The Town Red,Doja Cat


In [4]:
#Function to recommend song
def song_recommender(df1, df2):
    song = input("Please enter the name of a song: ")
    # Check if the input contains at least one alphabetical character and/or alphanumeric character
    if not (any(c.isalpha() for c in song) and any(c.isalnum() for c in song)):
        print("Invalid input. Please enter a song name containing letters and/or numbers.")
        return
    print(f"You entered a valid song name: {song}")
    # Load Spotify secrets
    secrets_file = open("secrets.txt", "r")
    string = secrets_file.read()
    secrets_dict = {}
    for line in string.split('\n'):
        if len(line) > 0:
            secrets_dict[line.split(':')[0]] = line.split(':')[1].strip()
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                               client_secret=secrets_dict['clientsecret']))
    results = sp.search(q='track:' + song, type='track')
    track = pd.json_normalize(results["tracks"]["items"])
    if not track.empty:
        song_id = track.iloc[0]['id']
        song_url = track.iloc[0]['external_urls.spotify']
    else:
        print("Song not found in Spotify.")
        return
    audio_features_list = sp.audio_features(song_id)
    audio_features_df = pd.json_normalize(audio_features_list)
    audio_features_df = audio_features_df[['duration_ms', 'danceability',
                                           'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
                                           'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]
    # Load scaler and transform data
    with open('scaler.pkl', 'rb') as f:
        loaded_scaler = pickle.load(f)
        X = loaded_scaler.transform(audio_features_df)
    # Load KMeans model and predict cluster
    with open('kmeans.pkl', 'rb') as f:
        loaded_kmeans = pickle.load(f)
        clusters = loaded_kmeans.predict(X)
    matching_song = df1[(df1['title'].str.lower() == song.lower())]
    if not matching_song.empty:
        print(f"The song '{matching_song['title'].values[0]}' by {matching_song['artist'].values[0]} matches your input.")
        recommended_song = df1[df1['title'].str.lower() != song].sample(1)
        print(f"Here's a recommended song: '{recommended_song['title'].values[0]}' by {recommended_song['artist'].values[0]}")
        results2 = sp.search(q='track:' + recommended_song['Title'].values[0], type='track')
        track2 = pd.json_normalize(results2["tracks"]["items"])
        song_url = track2.iloc[0]['external_urls.spotify']
        print(f"Here's a url of the song: {song_url}")
    else:
        cluster_df = df2[df2['clusters'] == clusters[0]]
        recommended_song2 = cluster_df[cluster_df['track_name'].str.lower() != song].sample(1)
        print(f"Here's a recommended song: '{recommended_song2['track_name'].values[0]}' by {recommended_song2['artists'].values[0]}")
        results3 = sp.search(q='track:' + recommended_song2['track_name'].values[0], type='track')
        track3 = pd.json_normalize(results3["tracks"]["items"])
        song_url = track3.iloc[0]['external_urls.spotify']
        print(f"Here's a url of the song: {song_url}")

In [5]:
song_recommender(df1, df2)

Please enter the name of a song: December
You entered a valid song name: December


/Users/heloisabal/anaconda3/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Here's a recommended song: 'Odawa' by Omar S
Here's a url of the song: https://open.spotify.com/track/3wlwwapt8U9PPR33s1ESR4


In [10]:
def recommender(df1, df2):
    song = input("Please enter the name of a song: ")
    # Check if the input contains at least one alphabetical character and/or alphanumeric character
    if not (any(c.isalpha() for c in song) and any(c.isalnum() for c in song)):
        print("Invalid input. Please enter a song name containing letters and/or numbers.")
        return
    print(f"You entered a valid song name: {song}")
    while True:  # This loop will continue to run until "ok" is entered by the user
        # Load Spotify secrets
        secrets_file = open("secrets.txt", "r")
        string = secrets_file.read()
        secrets_dict = {}
        for line in string.split('\n'):
            if len(line) > 0:
                secrets_dict[line.split(':')[0]] = line.split(':')[1].strip()
        sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                               client_secret=secrets_dict['clientsecret']))
        results = sp.search(q='track:' + song, type='track')
        track = pd.json_normalize(results["tracks"]["items"])
        if not track.empty:
            song_id = track.iloc[0]['id']
            song_url = track.iloc[0]['external_urls.spotify']
        else:
            print("Song not found in Spotify.")
            return
        audio_features_list = sp.audio_features(song_id)
        audio_features_df = pd.json_normalize(audio_features_list)
        audio_features_df = audio_features_df[['duration_ms', 'danceability',
                                           'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
                                           'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]
        # Load scaler and transform data
        with open('scaler.pkl', 'rb') as f:
            loaded_scaler = pickle.load(f)
            X = loaded_scaler.transform(audio_features_df)
        # Load KMeans model and predict cluster
        with open('kmeans.pkl', 'rb') as f:
            loaded_kmeans = pickle.load(f)
            clusters = loaded_kmeans.predict(X)
        matching_song = df1[(df1['title'].str.lower() == song.lower())]
        if not matching_song.empty:
            print(f"The song '{matching_song['title'].values[0]}' by {matching_song['artist'].values[0]} matches your input.")
            recommended_song = df1[df1['title'].str.lower() != song].sample(1)
            print(f"Here's a recommended song: '{recommended_song['title'].values[0]}' by {recommended_song['artist'].values[0]}")
            results2 = sp.search(q='track:' + recommended_song['title'].values[0], type='track')
            track2 = pd.json_normalize(results2["tracks"]["items"])
            song_url = track2.iloc[0]['external_urls.spotify']
            print(f"Here's a url of the song: {song_url}")
        else:
            cluster_df = df2[df2['clusters'] == clusters[0]]
            recommended_song2 = cluster_df[cluster_df['track_name'].str.lower() != song].sample(1)
            print(f"Here's a recommended song: '{recommended_song2['track_name'].values[0]}' by {recommended_song2['artists'].values[0]}")
            results3 = sp.search(q='track:' + recommended_song2['track_name'].values[0], type='track')
            track3 = pd.json_normalize(results3["tracks"]["items"])
            song_url = track3.iloc[0]['external_urls.spotify']
            print(f"Here's a url of the song: {song_url}")
        user_feedback = input("If you want another recommendation, type 'next'. If you're satisfied, type 'ok': ").strip().lower()
        if user_feedback == 'ok':
            break
        elif user_feedback == 'next':
            continue
        else:
            print("Invalid input. Please type 'next' or 'ok'.")

In [ ]:
recommender(df1, df2)

Please enter the name of a song: Hello
You entered a valid song name: Hello


/Users/heloisabal/anaconda3/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Here's a recommended song: 'Joy' by Apollo 100
Here's a url of the song: https://open.spotify.com/track/4EuaDcxHmHKS6gZ5iuGba0
